### Spark HW2 Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [3]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
# need to first download dataset from MovieLens, unzip files, and upload csv files to Databricks
# before executing the lines below.
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [7]:
movies_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
ratings_df.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [9]:
links_df.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [10]:
tags_df.show(5)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [11]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1

In [12]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

## Part 1: Spark SQL and OLAP

In [14]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Q1: The number of Users

In [16]:
%sql 

with cte as
(
  select count(*) as num_movies_rated, userID
  from ratings
  group by userID
)

select count(userID) as number_of_users from cte;

number_of_users
610


### Q2: The number of Movies

In [18]:
%sql

with cte as
(
  select count(*), movieID
  from movies
  group by movieID
)

select count(movieID) as number_of_movies from cte;

number_of_movies
9742


### Q3:  How many movies are rated by users? List movies not rated before

In [20]:
%sql

with cte as
(
  select movies.movieID as ratedMovieID
  from ratings join movies on ratings.movieID = movies.movieID
)

select count(distinct ratedMovieID) as num_of_rated_movies from cte;

num_of_rated_movies
9724


there are 9742 movies in total, and 9724 of them have been rated. So there are 18 movies that haven't been rated 
by anyone. They are listed below

In [22]:
%sql

select title as movies_not_rated_before, genres
from movies
where movieID not in (select movieID from ratings)
order by title

movies_not_rated_before,genres
"Browning Version, The (1951)",Drama
Call Northside 777 (1948),Crime|Drama|Film-Noir
Chalet Girl (2011),Comedy|Romance
"Chosen, The (1981)",Drama
"Color of Paradise, The (Rang-e khoda) (1999)",Drama
For All Mankind (1989),Documentary
I Know Where I'm Going! (1945),Drama|Romance|War
In the Realms of the Unreal (2004),Animation|Documentary
"Innocents, The (1961)",Drama|Horror|Thriller
Mutiny on the Bounty (1962),Adventure|Drama|Romance


### Q4: List Movie Genres

In [24]:
display(movies_df)

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


In [25]:
import pyspark.sql.functions as f

In [26]:
# select out all the distinct genres as another dataframe 
# by using posexplode function.
genres_df = movies_df.select(
        "title",
        f.posexplode(f.split("genres", "\|")).alias("index", "genre")
    )

In [27]:
display(genres_df)

title,index,genre
Toy Story (1995),0,Adventure
Toy Story (1995),1,Animation
Toy Story (1995),2,Children
Toy Story (1995),3,Comedy
Toy Story (1995),4,Fantasy
Jumanji (1995),0,Adventure
Jumanji (1995),1,Children
Jumanji (1995),2,Fantasy
Grumpier Old Men (1995),0,Comedy
Grumpier Old Men (1995),1,Romance


In [28]:
genres_df.select("genre").distinct().show()

+------------------+
 genre|
+------------------+
 Crime|
 Romance|
 Thriller|
 Adventure|
 Drama|
 War|
 Documentary|
 Fantasy|
 Mystery|
 Musical|
 Animation|
 Film-Noir|
(no genres listed)|
 IMAX|
 Horror|
 Western|
 Comedy|
 Children|
 Action|
 Sci-Fi|
+------------------+

### Q5: Movie for Each Category

In [30]:
genres_df.groupby("genre").count().orderBy("count", ascending = False).show()

+------------------+-----+
 genre|count|
+------------------+-----+
 Drama| 4361|
 Comedy| 3756|
 Thriller| 1894|
 Action| 1828|
 Romance| 1596|
 Adventure| 1263|
 Crime| 1199|
 Sci-Fi| 980|
 Horror| 978|
 Fantasy| 779|
 Children| 664|
 Animation| 611|
 Mystery| 573|
 Documentary| 440|
 War| 382|
 Musical| 334|
 Western| 167|
 IMAX| 158|
 Film-Noir| 87|
(no genres listed)| 34|
+------------------+-----+

In [31]:
genres_to_movies_df = genres_df.groupby("genre").agg(f.collect_list("title").alias("movies of this genre"))

In [32]:
display(genres_to_movies_df)

genre movies of this genre Crime List(Heat (1995), Casino (1995), Money Train (1995), Get Shorty (1995), Copycat (1995), Assassins (1995), Shanghai Triad (Yao a yao yao dao waipo qiao) (1995), Dead Man Walking (1995), Dead Presidents (1995), Usual Suspects, The (1995), Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996), Crossing Guard, The (1995), Things to Do in Denver When You're Dead (1995), Hate (Haine, La) (1995), Bottle Rocket (1996), Taxi Driver (1976), Rumble in the Bronx (Hont faan kui) (1995), Young Poisoner's Handbook, The (1995), Bad Boys (1995), Amateur (1994), Batman Forever (1995), Clockers (1995), Devil in a Blue Dress (1995), Die Hard: With a Vengeance (1995), Doom Generation, The (1995), Hackers (1995), Judge Dredd (1995), Net, The (1995), Strange Days (1995), Bushwhacked (1995), Heavenly Creatures (1994), Kiss of Death (1995), New Jersey Drive (1995), New York Cop (Nyû Yôku no koppu) (1993), Natural Born Killers (1994), Once Were Warriors (1994), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Pulp Fiction (1994), Shawshank Redemption, The (1994), Tales from the Hood (1995), Clear and Present Danger (1994), Crow, The (1994), Mask, The (1994), Bad Company (1995), Faster Pussycat! Kill! Kill! (1965), Jason's Lyric (1994), Above the Rim (1994), Beverly Hills Cop III (1994), Carlito's Way (1993), Fresh (1994), Getaway, The (1994), Hard Target (1993), Jimmy Hollywood (1994), Judgment Night (1993), Killing Zoe (1994), Menace II Society (1993), Perfect World, A (1993), RoboCop 3 (1993), Romeo Is Bleeding (1993), Serial Mom (1994), Striking Distance (1993), True Romance (1993), Escort, The (Scorta, La) (1993), Batman (1989), Silence of the Lambs, The (1991), Love and a .45 (1994), Fargo (1996), Primal Fear (1996), Courage Under Fire (1996), Substitute, The (1996), Mulholland Falls (1996), Heaven's Prisoners (1996), Striptease (1996), Trainspotting (1996), American Buffalo (1996), Ransom (1996), Godfather, The (1972), Supercop (Police Story 3: Supercop) (Jing cha gu shi III: Chao ji jing cha) (1992), Bound (1996), Carpool (1996), Kansas City (1996), Nothing to Lose (1994), Supercop 2 (Project S) (Chao ji ji hua) (1993), Bulletproof (1996), Some Like It Hot (1959), Charade (1963), To Catch a Thief (1955), Laura (1944), Thin Man, The (1934), Beat the Devil (1953), Last Man Standing (1996), Set It Off (1996), 2 Days in the Valley (1996), Freeway (1996), Die Hard (1988), Normal Life (1996), Crossfire (1947), Fish Called Wanda, A (1988), Bonnie and Clyde (1967), Dial M for Murder (1954), Reservoir Dogs (1992), Basic Instinct (1992), Funeral, The (1996), Return of the Pink Panther, The (1975), Wallace & Gromit: The Wrong Trousers (1993), Grifters, The (1990), Tie Me Up! Tie Me Down! (¡Átame!) (1990), Clockwork Orange, A (1971), Goodfellas (1990), Killer, The (Die xue shuang xiong) (1989), Psycho (1960), Godfather: Part II, The (1974), Once Upon a Time in America (1984), Sting, The (1973), Miller's Crossing (1990), Touch of Evil (1958), Femme Nikita, La (Nikita) (1990), Chinatown (1974), M (1931), Manchurian Candidate, The (1962), Fried Green Tomatoes (1991), Big Sleep, The (1946), Cape Fear (1962), Blood and Wine (Blood & Wine) (1996), Albino Alligator (1996), Batman Returns (1992), Sneakers (1992), Beavis and Butt-Head Do America (1996), La Cérémonie (1995), Hamlet (1996), Thieves (Voleurs, Les) (1996), Murder at 1600 (1997), Metro (1997), Gridlock'd (1997), Lost Highway (1997), Donnie Brasco (1997), Best Men (1997), Grosse Pointe Blank (1997), Face/Off (1997), Cop Land (1997), Desperate Measures (1998), Hoodlum (1997), L.A. Confidential (1997), Kiss the Girls (1997), U Turn (1997), Playing God (1997), Switchback (1997), Gang Related (1997), Deceiver (1997), Incognito (1997), Red Corner (1997), Man Who Knew Too Little, The (1997), Midnight in the Garden of Good and Evil (1997), Jackie Brown (1997), Big Lebowski, The (1998), Hard Rain (1998), Fallen (1998), Replacement Killers, 

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [34]:
ratings_df.show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

In [35]:
movie_ratings=ratings_df.drop('timestamp')

In [36]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [37]:
movie_ratings.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
 1| 70| 3.0|
 1| 101| 5.0|
 1| 110| 4.0|
 1| 151| 5.0|
 1| 157| 5.0|
 1| 163| 5.0|
 1| 216| 5.0|
 1| 223| 3.0|
 1| 231| 5.0|
 1| 235| 4.0|
 1| 260| 5.0|
 1| 296| 3.0|
 1| 316| 3.0|
 1| 333| 5.0|
 1| 349| 4.0|
+------+-------+------+
only showing top 20 rows

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [39]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [40]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.7,0.3])

In [41]:
#Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [42]:
#initialize ParamGridBuilder
paramGrid = ParamGridBuilder()\
  .addGrid(als.rank, [10, 50])\
  .addGrid(als.regParam, [0.1, 0.01])\
  .addGrid(als.maxIter, [5, 10])\
  .build()

In [43]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse", labelCol="rating", predictionCol="prediction")

In [44]:
# Build Cross validator
crossval = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

In [45]:
#Run cross-validation of ALS model on training set with grid search on hyperparameters.
cvModel = crossval.fit(training)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [46]:
#Extract best model from the grid search
best_model = cvModel.bestModel

### Model testing
And finally, make a prediction and check the testing error.

In [48]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [49]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:" + str(best_model._java_obj.parent().getRank())), 
print (" MaxIter:" + str(best_model._java_obj.parent().getMaxIter())), 
print (" RegParam:" + str(best_model._java_obj.parent().getRegParam())), 

RMSE = 0.8972249244124295
**Best Model**
 Rank:50
 MaxIter:10
 RegParam:0.1
Out[101]: (None,)

In [50]:
predictions.show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 133| 471| 4.0| 2.5021684|
 597| 471| 2.0| 3.7761111|
 602| 471| 4.0| 3.0759747|
 372| 471| 3.0| 3.1841335|
 57| 471| 3.0| 3.118896|
 610| 471| 4.0| 3.5063262|
 217| 471| 2.0| 2.6852248|
 520| 471| 5.0| 3.4329047|
 216| 471| 3.0| 3.281883|
 287| 471| 4.5| 2.2712877|
 469| 471| 5.0| 3.5358615|
 426| 471| 5.0| 3.3564987|
 541| 471| 3.0| 3.3250763|
 373| 471| 5.0| 2.473968|
 104| 471| 4.5| 3.155565|
 44| 833| 2.0| 2.5030432|
 307| 833| 1.0| 1.998311|
 608| 833| 0.5| 2.486876|
 606| 1088| 3.0| 3.2973423|
 132| 1088| 4.0| 2.9625552|
+------+-------+------+----------+
only showing top 20 rows

### Model apply and see the performance

In [52]:
alldata= best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = " + str(rmse))

RMSE = 0.6063475156508589

In [53]:
alldata.registerTempTable("alldata")

In [54]:
%sql select * from alldata

userId,movieId,rating,prediction
191,148,5.0,4.9190273
133,471,4.0,2.5021684
597,471,2.0,3.7761111
385,471,4.0,3.5130453
436,471,3.0,3.2974617
602,471,4.0,3.0759747
91,471,1.0,2.0357997
409,471,3.0,3.2527194
372,471,3.0,3.1841335
599,471,2.5,2.819613


In [55]:
%sql select * from movies join alldata on movies.movieId=alldata.movieId

movieId,title,genres,userId,movieId,rating,prediction
148,"Awfully Big Adventure, An (1995)",Drama,191,148,5.0,4.9190273
471,"Hudsucker Proxy, The (1994)",Comedy,133,471,4.0,2.5021684
471,"Hudsucker Proxy, The (1994)",Comedy,597,471,2.0,3.7761111
471,"Hudsucker Proxy, The (1994)",Comedy,385,471,4.0,3.5130453
471,"Hudsucker Proxy, The (1994)",Comedy,436,471,3.0,3.2974617
471,"Hudsucker Proxy, The (1994)",Comedy,602,471,4.0,3.0759747
471,"Hudsucker Proxy, The (1994)",Comedy,91,471,1.0,2.0357997
471,"Hudsucker Proxy, The (1994)",Comedy,409,471,3.0,3.2527194
471,"Hudsucker Proxy, The (1994)",Comedy,372,471,3.0,3.1841335
471,"Hudsucker Proxy, The (1994)",Comedy,599,471,2.5,2.819613


## Recommend moive to users with id: 575, 232. 
you can choose some users to recommend the moives

In [57]:
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllUsers(10)

In [58]:
# first let's do recommendation for user whose id is 232.
movies_rec_for_user_232 = movieRecs[movieRecs.userId.isin(232)]

In [59]:
display(movies_rec_for_user_232)

userId,recommendations
232,"List(List(120478, 4.3886175), List(2131, 4.3753724), List(78836, 4.357968), List(171011, 4.3255415), List(177593, 4.304807), List(147250, 4.28212), List(130970, 4.28212), List(174551, 4.28212), List(142020, 4.28212), List(104780, 4.28212))"


In [60]:
movies_232 = movies_rec_for_user_232.select(
        "userId",
        f.posexplode("recommendations").alias("index", "recommendation")
    )

In [61]:
display(movies_232)

userId,index,recommendation
232,0,"List(120478, 4.3886175)"
232,1,"List(2131, 4.3753724)"
232,2,"List(78836, 4.357968)"
232,3,"List(171011, 4.3255415)"
232,4,"List(177593, 4.304807)"
232,5,"List(147250, 4.28212)"
232,6,"List(130970, 4.28212)"
232,7,"List(174551, 4.28212)"
232,8,"List(142020, 4.28212)"
232,9,"List(104780, 4.28212)"


In [62]:
movies_232 = movies_232.withColumn("movieId", col("recommendation")["movieId"])

In [63]:
movies_232 = movies_232.withColumn("rating", col("recommendation")["rating"])

In [64]:
movies_232.show()

+------+-----+-------------------+-------+---------+
userId|index| recommendation|movieId| rating|
+------+-----+-------------------+-------+---------+
 232| 0|[120478, 4.3886175]| 120478|4.3886175|
 232| 1| [2131, 4.3753724]| 2131|4.3753724|
 232| 2| [78836, 4.357968]| 78836| 4.357968|
 232| 3|[171011, 4.3255415]| 171011|4.3255415|
 232| 4| [177593, 4.304807]| 177593| 4.304807|
 232| 5| [147250, 4.28212]| 147250| 4.28212|
 232| 6| [130970, 4.28212]| 130970| 4.28212|
 232| 7| [174551, 4.28212]| 174551| 4.28212|
 232| 8| [142020, 4.28212]| 142020| 4.28212|
 232| 9| [104780, 4.28212]| 104780| 4.28212|
+------+-----+-------------------+-------+---------+

In [65]:
movies_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [66]:
movies_232 = movies_232.join(movies_df, "movieId")

In [67]:
display(movies_232)

movieId,userId,index,recommendation,rating,title,genres
120478,232,0,"List(120478, 4.3886175)",4.3886175,The Salt of the Earth (2014),Documentary
2131,232,1,"List(2131, 4.3753724)",4.3753724,Autumn Sonata (Höstsonaten) (1978),Drama
78836,232,2,"List(78836, 4.357968)",4.357968,Enter the Void (2009),Drama
171011,232,3,"List(171011, 4.3255415)",4.3255415,Planet Earth II (2016),Documentary
177593,232,4,"List(177593, 4.304807)",4.304807,"Three Billboards Outside Ebbing, Missouri (2017)",Crime|Drama
147250,232,5,"List(147250, 4.28212)",4.28212,The Adventures of Sherlock Holmes and Doctor Watson,(no genres listed)
130970,232,6,"List(130970, 4.28212)",4.28212,George Carlin: Life Is Worth Losing (2005),Comedy
174551,232,7,"List(174551, 4.28212)",4.28212,Obsession (1965),Comedy
142020,232,8,"List(142020, 4.28212)",4.28212,Oscar (1967),Comedy
104780,232,9,"List(104780, 4.28212)",4.28212,"Mystery of the Third Planet, The (Tayna tretey planety) (1981)",Adventure|Animation|Sci-Fi


In [68]:
ratings_df.show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

In [69]:
# we want to filter out the movies that user 232 already watched, and
# only recommend the movies that he/she hasn't watched.
# first let's find the movies that user 232 has watched.
has_watched_by_232 = ratings_df.select("movieId").where(ratings_df['userId'] == 232)

In [70]:
has_watched_by_232.show()

+-------+
movieId|
+-------+
 1|
 2|
 10|
 39|
 47|
 48|
 104|
 110|
 145|
 150|
 165|
 185|
 231|
 258|
 260|
 296|
 303|
 317|
 318|
 356|
+-------+
only showing top 20 rows

In [71]:
# now we select the movies suggested by recommendatoin system
# and have not been watched by this user
# don't know how to do this in spark dataframe...
# will first convert these dataframes into sql tables.
movies_232.registerTempTable("movies_232")
has_watched_by_232.registerTempTable("has_watched_by_232")

In [72]:
%sql

--- the following movies are what I will recommend for user 232.
select userId, title, rating
from movies_232
where movieId not in 
  (select movieId from has_watched_by_232)

userId,title,rating
232,The Salt of the Earth (2014),4.3886175
232,Autumn Sonata (Höstsonaten) (1978),4.3753724
232,Enter the Void (2009),4.357968
232,Planet Earth II (2016),4.3255415
232,"Three Billboards Outside Ebbing, Missouri (2017)",4.304807
232,The Adventures of Sherlock Holmes and Doctor Watson,4.28212
232,George Carlin: Life Is Worth Losing (2005),4.28212
232,Obsession (1965),4.28212
232,Oscar (1967),4.28212
232,"Mystery of the Third Planet, The (Tayna tretey planety) (1981)",4.28212


In [73]:
# now let's recommend for user 575
movies_rec_for_user_575 = movieRecs[movieRecs.userId.isin(575)]

In [74]:
movies_575 = movies_rec_for_user_575.select(
        "userId",
        f.posexplode("recommendations").alias("index", "recommendation")
    )

In [75]:
movies_575 = movies_575.withColumn("movieId", col("recommendation")["movieId"])

In [76]:
movies_575 = movies_575.withColumn("rating", col("recommendation")["rating"])

In [77]:
movies_575 = movies_575.join(movies_df, "movieId")

In [78]:
has_watched_by_575 = ratings_df.select("movieId").where(ratings_df['userId'] == 575)

In [79]:
movies_575.registerTempTable("movies_575")
has_watched_by_575.registerTempTable("has_watched_by_575")

In [80]:
%sql

--- the following movies are what I will recommend for user 575.
select userId, title, rating
from movies_575
where movieId not in 
  (select movieId from has_watched_by_575)

userId,title,rating
575,"Three Billboards Outside Ebbing, Missouri (2017)",5.008256
575,"Timecrimes (Cronocrímenes, Los) (2007)",4.694389
575,Mary and Max (2009),4.589497
575,"Jetée, La (1962)",4.5804543
575,"Departed, The (2006)",4.571027
575,Incendies (2010),4.5592093
575,Fight Club (1999),4.552477


## Find the similar moives for moive with id: 463, 471
You can find the similar moives based on the ALS results

In [82]:
# what is the name of our movie with id 471?
display(movies_df.select("title").where(movies_df.movieId == 471))

title
"Hudsucker Proxy, The (1994)"


In [83]:
# what is the name of our movie with id 463?
display(movies_df.select("title").where(movies_df.movieId == 463))

title


In [84]:
# we don't actually have a movie with id 463.
# so we will only try to find similar movies to movie 463

In [85]:
# get vector representation of movies.
movies_features = best_model.itemFactors

In [86]:
display(movies_features)

id,features
10,"List(0.052700475, -0.1386982, -0.3289289, -0.31805393, 0.08622593, -0.07868539, -0.15322477, -0.19596267, 0.2674041, -0.354641, -0.17032628, 0.54862547, 0.047402952, -0.030444833, 0.7103106, 0.03844292, -0.50558734, 0.068401024, -0.12167111, 0.20623939, -0.35683644, -0.24759585, 0.1816506, -0.4219719, 0.116659075, 0.7603133, 0.08416081, -0.12562725, -0.29395434, -0.32652602, 0.3262476, 0.0092290165, 0.041243047, 0.21058607, -0.23293617, 0.14697546, 0.2630483, 0.32540223, -0.21415342, -0.6848289, -0.41369826, 0.036219943, 0.16801606, -0.28578794, 0.5870047, -0.045512833, 0.24064219, -0.09731782, -0.37595335, -0.15876092)"
20,"List(-0.2927772, -0.070501186, -0.3400771, -0.119965516, 0.23522629, -0.054070763, -0.07599353, -0.20040375, 0.15477012, -0.07809445, -0.19725934, 0.36223543, 0.26233113, -0.021590445, 0.108508505, -0.0039723786, -0.04681693, -0.15912668, -0.49367574, -0.040563397, -0.49151716, -0.18492354, 0.37809572, -0.07464735, -0.021229526, 0.37459293, 0.28413752, -0.38366538, -0.012576112, -0.55840486, 0.31000912, 0.1925429, -0.11498306, 0.22018908, 0.08336152, 0.18859766, 0.054118033, 0.30788186, 0.092742585, -0.5458834, -0.40300637, -0.23930399, 0.02463199, -0.23913364, 0.13104813, 0.25121635, 0.033840846, -0.20744838, -0.18117695, 0.12296344)"
30,"List(-0.4694317, -0.19216359, -0.26447412, -0.088832945, 0.033741616, 0.008202704, 0.017173208, 0.16183376, 0.1078591, -0.6572303, -0.23657596, 0.33815208, -0.0385073, 0.15914269, 0.4005916, -0.2675402, -0.14711864, 0.08123174, -0.105402105, 0.28789267, -0.25546464, -0.20975985, 0.08050195, -0.009350141, 0.2300314, 0.825588, 0.08197414, -0.07357901, -0.039175484, -0.39944935, 0.039324414, 0.2309389, 0.05819447, 0.10950709, -0.102732286, -0.23950198, 0.47055808, 0.2449109, 0.16692635, -0.57071793, -0.24466449, -0.072903514, -0.26857013, -0.23914108, 0.5496522, 0.24078324, 0.1001634, -0.17403936, 0.017078456, 0.107138515)"
40,"List(-0.15071489, -0.3599208, -0.17190614, -0.29822907, 0.19665529, 0.5487628, 0.098867945, 0.33640322, -0.13102832, -0.25392637, -0.19703235, 0.041172918, 0.25496337, 0.18142334, 0.644229, 0.030859478, 0.043743633, -0.07524604, -0.4187842, 0.16638787, -0.30324823, -0.21338138, -0.0528477, -0.21873191, -0.003529133, 0.45616794, 0.12388077, 0.26054183, -0.44516492, -0.47271267, 0.20701088, 0.026724076, -0.15418355, 0.34391806, 0.01647039, 0.16432841, 0.4780543, 0.6684879, 0.1620969, -0.5015844, -0.42455417, 0.12277934, -0.2497955, -0.28498563, 0.23740081, 0.13570917, 0.024523098, -0.38540858, -0.2337974, 0.021041024)"
50,"List(-0.5139816, -0.3791663, -0.49517345, -0.10969815, 0.44384113, 0.07691143, -0.085174784, 0.48562905, -0.13039264, -0.36896157, -0.54579806, 0.42052594, 0.14433779, -0.08433656, 0.67200613, -0.1873005, -0.1623905, 0.19374146, -0.2800523, 0.3204323, -0.12332596, -0.41134512, 0.28508162, -0.42769468, 0.10657681, 0.7214842, 0.25517488, -0.05223179, -0.1372096, -0.53207326, 0.18710725, 0.40146047, 0.044078454, 0.12756574, 0.05286967, 0.3833586, 0.6091164, 0.34660727, 0.16082262, -0.72094804, -0.23344362, -0.07263578, 0.049630966, -0.43364093, 0.5624848, 0.029003475, 0.1944369, -0.27923557, -0.21183628, 0.27337185)"
60,"List(-0.2613401, -0.15216587, 0.097305425, 0.023669269, 0.3779707, -0.0756864, 0.021689525, 0.39459264, 0.2496192, -0.61738425, -0.25063947, 0.092416644, -0.31258258, 0.16252944, 0.38760412, 0.17964272, -0.28451192, 0.039532486, -0.3221422, 0.103648886, -0.1182344, -0.33248255, -0.017436827, -0.32950935, -0.1493111, 0.3831251, 0.23524846, -0.06537785, -0.27205116, -0.38145447, 0.48861232, 0.09792493, 0.15743215, 0.22247316, 0.14382876, -0.077524, 0.101506494, 0.3221132, -0.14779238, -0.53569835, -0.58600974, -0.12743694, -0.20144254, -0.3485225, 0.36364123, 0.23162954, 0.5307832, 0.3580458, -0.55648935, 0.107940815)"
70,"List(-0.07495977, 0.056525886, -0.27558038, -0.07895627, 0.6331011, 0.03653169, 0.024840696, -0.14769316, -0.10519021, -0.4596661, -0.1277601, 0.3668839, 0.0532277, -0.074974686, 0.52791

In [87]:
movies_features.printSchema()

root
-- id: integer (nullable = false)
-- features: array (nullable = true)
 |-- element: float (containsNull = false)

In [88]:
# feature vector of movie with id 471
display(movies_features[movies_features.id == 471])

id,features
471,"List(-0.442949, -0.2190775, -0.05426431, 0.13043371, -0.09815314, 0.057117824, -0.60431886, 0.41558823, 0.30879933, -0.7790108, -0.45816347, 0.63097423, 0.33095533, -0.16324875, 0.42525232, 0.18617235, -0.15094194, 0.020421784, -0.005300771, 0.01826917, 0.25979114, 0.031285178, 0.4893201, -0.30733746, 0.3066172, 0.822976, 0.060549654, -0.31299117, -0.28012604, -0.20180291, 0.41699606, 0.027473077, 0.2094624, 0.19438918, -0.03157235, 0.11277115, 0.5778799, 0.14997472, 0.09418211, -0.15594567, -0.7222569, 0.13451435, -0.26450163, -0.33725262, 0.48486328, 0.3935318, -0.12470119, 0.04106205, -0.15055008, 0.12587292)"


In [89]:
# get our feature vector of movie 471 as a list.
feature_vector_471 = movies_features.where(movies_features.id == 471).select('features').collect()[0][0]

In [90]:
feature_vector_471

Out[454]: [-0.44294899702072144,
 -0.21907749772071838,
 -0.054264310747385025,
 0.13043370842933655,
 -0.09815313667058945,
 0.057117823511362076,
 -0.6043188571929932,
 0.41558822989463806,
 0.3087993264198303,
 -0.7790107727050781,
 -0.45816347002983093,
 0.6309742331504822,
 0.3309553265571594,
 -0.16324874758720398,
 0.4252523183822632,
 0.18617235124111176,
 -0.15094193816184998,
 0.02042178437113762,
 -0.005300770979374647,
 0.018269170075654984,
 0.25979113578796387,
 0.031285177916288376,
 0.4893200993537903,
 -0.3073374629020691,
 0.30661720037460327,
 0.8229759931564331,
 0.06054965406656265,
 -0.3129911720752716,
 -0.28012603521347046,
 -0.20180290937423706,
 0.4169960618019104,
 0.027473077178001404,
 0.20946240425109863,
 0.19438917934894562,
 -0.03157234936952591,
 0.11277115345001221,
 0.5778799057006836,
 0.14997471868991852,
 0.09418211132287979,
 -0.15594567358493805,
 -0.7222568988800049,
 0.13451434671878815,
 -0.2645016312599182,
 -0.3372526168823242,
 0.48486328125,
 0.39353179931640625,
 -0.12470118701457977,
 0.04106204956769943,
 -0.15055008232593536,
 0.1258729249238968]

In [91]:
# to find the similar movies we first have to define
# similarity mathematically. In this case, I will 
# define similarity as the cosine similarities between
# two vectors that represent two movies.

# similarity of close to 1 between two movies means they are
# very similar, 0 means no similarity, -1 means they are 
# opposite of each other.
# vec1, vec2 are feature representations of movies.
def cosineSimilarity(vec1, vec2):
  return float( np.dot(vec1,vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)) )

In [92]:
# we are going to use udf to calculate cosine similarities
# between every movie and movie 471. first need import packages.
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import expr
from pyspark.sql.types import DoubleType

In [93]:
# an udf that calculate the cosine similarity between movie 471 and another movie.
similaity_func =  udf(lambda another_movie_vector: cosineSimilarity(another_movie_vector, feature_vector_471), DoubleType())

In [94]:
movies_features = movies_features.withColumn("similarity_score", similaity_func(col("features")))

In [95]:
# get the top 20 similar movies to movie 471, based on our similarity score.
top_20_similar_df = movies_features.where(movies_features.id != 471).select("id", "similarity_score").orderBy("similarity_score", ascending = False).limit(20)

In [96]:
display(top_20_similar_df)

id,similarity_score
4029,0.8246750762785373
6027,0.8159813077528227
2692,0.81498285379571
151,0.8134824744308271
3169,0.8106844391653383
497,0.8099654760916022
1066,0.8066785608138571
3862,0.8046315924340476
3078,0.8040805599588399
5316,0.8040639361692914


In [97]:
# we want movie names rather than their id's. so we do a join
top_20_similar_df = top_20_similar_df.join(movies_df, top_20_similar_df.id == movies_df.movieId)

In [98]:
display(top_20_similar_df)

id,similarity_score,movieId,title,genres
151,0.8134824744308271,151,Rob Roy (1995),Action|Drama|Romance|War
251,0.7971016218487865,251,"Hunted, The (1995)",Action
497,0.8099654760916022,497,Much Ado About Nothing (1993),Comedy|Romance
574,0.7959494478989719,574,Spanking the Monkey (1994),Comedy|Drama
1066,0.8066785608138571,1066,Shall We Dance (1937),Comedy|Musical|Romance
2482,0.7952786011597546,2482,Still Crazy (1998),Comedy|Romance
2692,0.81498285379571,2692,Run Lola Run (Lola rennt) (1998),Action|Crime
3078,0.8040805599588399,3078,Liberty Heights (1999),Drama
3169,0.8106844391653383,3169,The Falcon and the Snowman (1985),Crime|Drama|Thriller
3360,0.7972866846424101,3360,Hoosiers (a.k.a. Best Shot) (1986),Drama|Romance


In [99]:
top_20_similar_df = top_20_similar_df.select("title", "similarity_score").orderBy("similarity_score", ascending = False)

In [100]:
# now this is list of our top 20 similar movies to movie "Hudsucker Proxy, The (1994)"
display(top_20_similar_df)

title,similarity_score
State and Main (2000),0.8246750762785373
Dogfight (1991),0.8159813077528227
Run Lola Run (Lola rennt) (1998),0.81498285379571
Rob Roy (1995),0.8134824744308271
The Falcon and the Snowman (1985),0.8106844391653383
Much Ado About Nothing (1993),0.8099654760916022
Shall We Dance (1937),0.8066785608138571
About Adam (2000),0.8046315924340476
Liberty Heights (1999),0.8040805599588399
Enigma (2001),0.8040639361692914


## Now I want to try to use this ALS model to recommend movies for myself.
To do so, I need to first enter some ratings about the movies I liked in
the past into our current movies dataframe. I will first need to see
what kind of movies are in the dataset. I will see what the top 50 ratings
movies (with more than 50 people rating) are and rate 15 of them. and combine
my information with movies_df to create another ALS model.

In [102]:
from pyspark.sql import functions as F

# create a dataframe with each movie's average ratings.
movie_ids_with_avg_ratings_df = ratings_df.groupBy('movieId').agg(F.count(ratings_df.rating).alias("count"), F.avg(ratings_df.rating).alias("average"))
print('movie_ids_with_avg_ratings_df:')
movie_ids_with_avg_ratings_df.show(3, truncate=False)

movie_ids_with_avg_ratings_df:
+-------+-----+------------------+
movieId|count|average |
+-------+-----+------------------+
296 |307 |4.197068403908795 |
1090 |63 |3.984126984126984 |
115713 |28 |3.9107142857142856|
+-------+-----+------------------+
only showing top 3 rows

In [103]:
# create a dataframe with 50 movies which have been rated by more than
# 50 people, and which has the highest average ratings
movie_names_df = movie_ids_with_avg_ratings_df.join(movies_df, "movieId")
top_50_popular_movies = movie_names_df.where(movie_names_df['count'] > 50).orderBy("average", ascending = False).limit(50)
display(top_50_popular_movies)

movieId,count,average,title,genres
318,317,4.429022082018927,"Shawshank Redemption, The (1994)",Crime|Drama
858,192,4.2890625,"Godfather, The (1972)",Crime|Drama
2959,218,4.272935779816514,Fight Club (1999),Action|Crime|Drama|Thriller
1276,57,4.271929824561403,Cool Hand Luke (1967),Drama
750,97,4.268041237113402,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Comedy|War
904,84,4.261904761904762,Rear Window (1954),Mystery|Thriller
1221,129,4.25968992248062,"Godfather: Part II, The (1974)",Crime|Drama
48516,107,4.252336448598131,"Departed, The (2006)",Crime|Drama|Thriller
1213,126,4.25,Goodfellas (1990),Crime|Drama
912,100,4.24,Casablanca (1942),Drama|Romance


Now I will choose 15 movies from above to rate. Since no user_id == 0, I will use id 0 for my ratings

In [105]:
from pyspark.sql import Row
my_user_id = 0

# you will notice I don't have any below 3 ratings.. That
# is just my style of ratings I guess. I never gave out 
# rating 1, even if it is terrible movie. 
my_rated_movies = [
  # The format of each line is (my_user_id, movie ID, your rating)
  # For example, to give the movie "Star Wars: Episode IV - A New Hope (1977)" a five rating, we will add:
  # (my_user_id, 260, 5)
  (my_user_id, 318, 4), # I like "The Shawshank Redemption"
  (my_user_id, 858, 3), # Godfather is okay
  (my_user_id, 58559, 5), # The Dark Knight! I loved it
  (my_user_id, 1197, 5), # I love The Princess Bride!!!
  (my_user_id, 260, 5), # I love star wars movies
  (my_user_id, 527, 4),
  (my_user_id, 1196, 5),
  (my_user_id, 2571, 4),
  (my_user_id, 356, 5),
  (my_user_id, 1136, 3),
  (my_user_id, 593, 4),
  (my_user_id, 5618, 5), #OMG I love Spirted Away!!!! Best animated movie
  (my_user_id, 1210, 5),
  (my_user_id, 78499, 3),
  (my_user_id, 4993, 4)
]

my_ratings_df = sqlContext.createDataFrame(my_rated_movies, ['userId','movieId','rating'])
print('My movie ratings:')
display(my_ratings_df.limit(10))

userId,movieId,rating
0,318,4
0,858,3
0,58559,5
0,1197,5
0,260,5
0,527,4
0,1196,5
0,2571,4
0,356,5
0,1136,3


Now I need to add my ratings to the training dataset

In [107]:
training_with_my_ratings_df = training.unionAll(my_ratings_df)

# make sure we have combined our dataframes right.
assert (training_with_my_ratings_df.count() - training.count()) == my_ratings_df.count()

Train ALS with my ratings in the dataset, with the best hyperparameters found earlier: 
MaxIter: 10,
RegParameter: 0.1,
Rank: 50

In [109]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", maxIter = 10, regParam = 0.1, rank = 50)

In [110]:
model = als.fit(training_with_my_ratings_df)

In [111]:
#Generate predictions and evaluate using RMSE on test set
predictions = model.transform(test)
rmse = evaluator.evaluate(predictions)
print ("RMSE on test set= "+str(rmse))

RMSE on test set= 0.8902463441457316

In [112]:
#Generate predictions and evaluate using RMSE on whole data set.
alldata= model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE overall = " + str(rmse))

RMSE overall = 0.6037253036508531

recommend 10 movies for myself!

In [114]:
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllUsers(30)
movies_rec_for_me = movieRecs[movieRecs.userId.isin(0)]

In [115]:
display(movies_rec_for_me)

userId,recommendations
0,"List(List(1210, 4.7010846), List(1197, 4.5842466), List(1196, 4.556859), List(260, 4.529009), List(58559, 4.5082088), List(177765, 4.4962726), List(5618, 4.4874105), List(318, 4.455045), List(356, 4.343371), List(1270, 4.3160996), List(3022, 4.2747035), List(899, 4.2250023), List(33166, 4.2204347), List(3451, 4.202602), List(1939, 4.1916165), List(7099, 4.1781063), List(3000, 4.1695056), List(954, 4.165675), List(1207, 4.161541), List(3213, 4.159915), List(92259, 4.1589093), List(1372, 4.1584888), List(81845, 4.1560993), List(8368, 4.1543245), List(115569, 4.1478314), List(166528, 4.1467943), List(26662, 4.146365), List(5952, 4.143963), List(912, 4.132637), List(89904, 4.131149))"


In [116]:
my_movies = movies_rec_for_me.select(
        "userId",
        F.explode("recommendations").alias("recommendation")
    )
my_movies = my_movies.withColumn("movieId", F.col("recommendation")["movieId"])
my_movies = my_movies.withColumn("rating", F.col("recommendation")["rating"])
my_movies = my_movies.join(movies_df, "movieId")
display(my_movies.select("userId", "title", "rating"))

userId,title,rating
0,Star Wars: Episode VI - Return of the Jedi (1983),4.7010846
0,"Princess Bride, The (1987)",4.5842466
0,Star Wars: Episode V - The Empire Strikes Back (1980),4.556859
0,Star Wars: Episode IV - A New Hope (1977),4.529009
0,"Dark Knight, The (2008)",4.5082088
0,Coco (2017),4.4962726
0,Spirited Away (Sen to Chihiro no kamikakushi) (2001),4.4874105
0,"Shawshank Redemption, The (1994)",4.455045
0,Forrest Gump (1994),4.343371
0,Back to the Future (1985),4.3160996


This is interesting! Besides the movies that I already rated, ALS gives me movies that I haven't watched before
but which I want to watch! Like "Back to the Future", and "Coco", and "Princess Mononoke". I will definitely check those out
when I have time! I will say the predictions are pretty accurate.

## Report/Analysis
motivation: My ultimate motivation is to use my newfound knowledges about recommendation systems and ALS model to recommend movies to watch in the future for myself.

steps:

1. I downloaded dataset about movie ratings from MovieLens website. I downloaded small dataset for the ease of training models. The dataset contains 100,000 ratings about 9,000 movies from 600 users. 
2. I uploaded these CSV files to my databricks filesystem and loaded them into spark dataframes.
3. I created SQL tables from these dataframes and used SQL queries to do some OLAP on the dataset, which tells me some insights: There are 9742 distinct movies in our dataset, and 610 users. There are 18 movies that haven't been rated by anyone. About a half movies in our dataset are of the genre drama/comedy. Also, there are about 15 genres each movie can be in, including Romance, thriller, drama, etc.
4. Then I used cross validation and grid search to find the best hyperparameters for our ALS model. It turns out the best rank for our ALS model is 50, and best regularization parameter is 0.1. I am quite surprised that best rank is 50 since that sounds like a lot of latent variables, but it is the best due to our cross validation, so I believe it.
5. the RMSE error of the best ALS model on test dataset is 0.897, and its RMSE error on the whole dataset is 0.606, which convinces me that our model hasn't overfitted since it generalizes well to the whole dataset.
6. With the trained ALS model, I tried to recommend movies for user 232 and user 575. Based on our model, I will recommend movies "The Salt of the Earth (2014)", "Autumn Sonata (Höstsonaten) (1978)", and "Enter the Void (2009)" to user 232. I will recommend movies "Three Billboards Outside Ebbing, Missouri (2017)", "Timecrimes (Cronocrímenes, Los) (2007)", and "Mary and Max (2009)" to user 575.
7. I also used cosine similarity to find the most similar movies to movie "Hudsucker Proxy, The (1994)". The top 3 similar ones are: "State and Main (2000)", "Dogfight (1991)", and "Run Lola Run (Lola rennt) (1998)", with greater than 0.8 similarity scores.
8. finally, I added my own movie ratings to the training dataset and trained ALS model again to predict what movies I would like to watch. It recommended for me to watch "Coco", "Back to the Future", and "Princess Mononoke", which I find surprising because those are the movies that I really want to watch but haven't got a chance to. It is amazing how accurate the model is.

skills learned from this project: cosine similarity, udf, explode function, ALS, tunning hyperparameters,...



[a notebook that taught me how to add my own ratings to training set](https://www.google.com)